In [2]:
import os

In [3]:
!pip install crewai crewai_tools langchain_community --upgrade

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.0/463.0 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.6 MB/s eta 0:00:00


In [5]:
import os
import yaml
from crewai import Agent, Task, Crew, LLM

In [6]:
!pip install litellm

In [16]:
import requests
from crewai import Agent, Task, Crew, Process
from langchain_core.tools import tool
from typing import Optional

In [15]:
# Configure LiteLLM for Azure
completion_model = {
    "model": "Phi-3-5-mini-instruct-iplvi",
    "api_type": "azure",
    "api_key": os.getenv("AZURE_API_KEY"),
    "api_base": os.getenv("AZURE_API_BASE"),
    "api_version": "2024-02-15-preview"  # Update this to your Azure API version
}

In [17]:
# Create the planner agent
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="""You're working on planning a blog article
              about the topic: {topic}.
              You collect information that helps the
              audience learn something
              and make informed decisions.
              Your work is the basis for
              the Content Writer to write an article on this topic.""",
    allow_delegation=False,
    llm_config=completion_model,  # Use llm_config instead of llm
    verbose=True
)

In [18]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    llm_config=completion_model,
    verbose=True
)

In [19]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    llm_config=completion_model,
    verbose=True
)

In [20]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [21]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [22]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [23]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

In [28]:
result = crew.kickoff(inputs={"topic": "Semicondutors"})

# Agent: Content Planner
## Task: Create a detailed content plan for an article about Semicondutors


# Agent: Content Planner
## Final Answer: 
Content Plan for Article: "Semiconductors: The Backbone of Modern Technology"

I. Introduction
   A. Definition of semiconductors
   B. Brief history of semiconductors
   C. Importance of semiconductors in today's world
   Potential Keywords: semiconductors, technology, history, importance

II. Understanding the Basics
   A. Types of semiconductors
      1. Intrinsic semiconductors
      2. Extrinsic semiconductors
   B. Key characteristics of semiconductors
      1. Energy bands
      2. Band gap
   C. The role of dopants
   Potential Keywords: intrinsic, extrinsic, energy bands, band gap, dopants

III. The Science Behind Semiconductors
   A. Electronic configuration of semiconductors
   B. Band theory and its application
   C. Carrier generation and recombination
   Potential Keywords: electronic configuration, band theory, carrier generatio

In [27]:
import os
from crewai import Agent, Task, Crew
from litellm import completion

# Set Azure OpenAI credentials
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://Phi-3-5-mini-instruct-iplvi.eastus2.models.ai.azure.com"
os.environ["OPENAI_API_VERSION"] = "2024-02-15-preview"  # Update this to your Azure API version

# Configure LiteLLM for Azure
completion_model = {
    "model": "Phi-3-5-mini-instruct-iplvi",
    "api_type": "azure",
    "api_key": os.getenv("OPENAI_API_KEY"),
    "api_base": os.getenv("OPENAI_API_BASE"),
    "api_version": os.getenv("OPENAI_API_VERSION")
}

# Create the planner agent
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="""You're working on planning a blog article
              about the topic: {topic}.
              You collect information that helps the
              audience learn something
              and make informed decisions.
              Your work is the basis for
              the Content Writer to write an article on this topic.""",
    allow_delegation=False,
    llm_config=completion_model,
    verbose=True
)

# Create a task
planning_task = Task(
    description="Create a detailed content plan for an article about {topic}",
    agent=planner,
    expected_output="A detailed content plan with sections, "
                    "subtopics, and potential keywords."
)

# Create the crew
crew = Crew(
    agents=[planner],
    tasks=[planning_task],
    verbose=True
)

# Run the crew
result = crew.kickoff(inputs={"topic": "Semiconductors"})
print(result)

# Agent: Content Planner
## Task: Create a detailed content plan for an article about Semiconductors


# Agent: Content Planner
## Final Answer: 
**Content Plan for Article: Semiconductors - The Building Blocks of the Modern World**

**1. Introduction**
   - Subtopic: Definition and Importance of Semiconductors
      - Keywords: semiconductors, modern technology, electronic components

   - Subtopic: Brief History of Semiconductors
      - Keywords: semiconductor history, development, milestones

**2. Basics of Semiconductors**
   - Subtopic: Atomic Structure and Electronic Properties
      - Keywords: atomic structure, electronic properties, semiconductor materials

   - Subtopic: Types of Semiconductors
      - Keywords: intrinsic, extrinsic, p-type, n-type

   - Subtopic: Band Theory and Charge Carriers
      - Keywords: band theory, electron, hole, conduction, valence

**3. Semiconductor Fabrication**
   - Subtopic: Materials Used in Semiconductor Manufacturing
      - Keywords: si